In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from aml_obs.collector import Online_Collector
from aml_obs.query import RT_Visualization
from azureml.core import Workspace
import pandas as pd

ws = Workspace.from_config()


### Real time metrics logging and visualization
#### You need to run this using your local computer or if you run with Azure ML Compute Instance, please do so from VSCode as Dash visualization requires access to localhost

In [ ]:
# Ingest streaming data  asynchronously with internal buffering mechanism to lower impact to main scoring thread
streaming_table_name="streaming_test"
streaming_collector = Online_Collector(streaming_table_name,ws=ws)

import random
streaming_collector.start_logging_daemon(buffer_time=2, batch_size=10)

for run_id in ["r000001", "r000002", "r000003", "r000004", "r000005"]:
    for i in range(1000):
        for lr in ["0.001", "0.002"]:
            df = pd.DataFrame({ "timestamp":pd.to_datetime('now'), "lr":[lr],"metric1":[random.uniform(3,50)] })
            streaming_collector.stream_collect_df_queue(df)
# streaming_collector.stop_logging_daemon()

In [ ]:
# from monitoring.query import RT_Visualization
streaming_table_name="streaming_test"
rt_viz =RT_Visualization(streaming_table_name,ws)
# rt_viz.scatter(max_records=200, ago='15m',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Model Monitoring

The deploy_model utility install AML CLI v2, create managed online endpoint, deploy the model. These steps are optional incase you want to supply your own names and objects. Check the function to see all paramters. The below just create everything.

In [ ]:
from deployment.deploy_util import deploy_model,score_test

# scoring_uri, scoring_key = deploy_model(ws=ws)

Once you have the model deployed with the collector object inside, you can just call it to simulate data creation. Data is collected to ADX table nIRIS_MODEL.

In [7]:
import time
scoring_uri="https://iris-ep.westus2.inference.ml.azure.com/score"
scoring_key="uPboOpl7zoiN5FVT2au3neWjnZCrp8oO"
for _ in range(200):
    time.sleep(3)
    score_test(scoring_uri, scoring_key)

b'["versicolor", "setosa", "virginica", "virginica", "virginica", "setosa", "versicolor", "versicolor", "setosa", "virginica"]'
b'["virginica", "setosa", "setosa", "virginica", "versicolor", "setosa", "setosa", "versicolor", "setosa", "virginica"]'
b'["setosa", "virginica", "versicolor", "virginica", "virginica", "setosa", "versicolor", "virginica", "versicolor", "virginica"]'
b'["virginica", "virginica", "virginica", "virginica", "setosa", "virginica", "versicolor", "setosa", "setosa", "versicolor"]'
b'["versicolor", "virginica", "virginica", "virginica", "setosa", "setosa", "virginica", "versicolor", "setosa", "virginica"]'
b'["setosa", "virginica", "virginica", "versicolor", "setosa", "versicolor", "setosa", "setosa", "setosa", "virginica"]'
b'["setosa", "versicolor", "virginica", "virginica", "versicolor", "setosa", "versicolor", "versicolor", "versicolor", "setosa"]'
b'["virginica", "virginica", "versicolor", "versicolor", "setosa", "setosa", "setosa", "setosa", "setosa", "virgini

### Monitoring Dashboard

Now that data is collected. Go to ADX Dashboard and import the quick_start/ADX_dashboards/databoard_Model_Monitoring.json to create a monitoring dashboard for your model.

Review https://docs.microsoft.com/en-us/azure/data-explorer/azure-data-explorer-dashboards#to-create-new-dashboard-from-a-file for instruction on how to import the dashboard.

You may need to correct a few default paramters such as connection to make dashboard work